In [3]:
!pip3 install textstat


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 175 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 22.9 MB/s eta 0:00:01
     |████████████████████████████████| 939 kB 4.1 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [8]:
# 📌 Step 1: Imports
import os
import json
from groq import Groq
from textstat import flesch_reading_ease

# 📌 Step 2: GROQ API setup
from dotenv import load_dotenv
load_dotenv()

# 📌 Step 1: Imports
import datetime
from groq import Groq

# 📌 Step 2: Setup GROQ API
api_key = os.getenv("GROQ_API_KEY")

# 📌 Step 3: Reward calculation function
def calculate_reward(original_text, new_text):
    # Length similarity
    orig_len = len(original_text)
    new_len = len(new_text)
    length_score = 1 - abs(orig_len - new_len) / max(orig_len, new_len)

    # Readability score
    readability = flesch_reading_ease(new_text)
    readability_score = min(readability / 100, 1)

    # Human-likeness using LLM
    prompt = f"Rate how human-like and coherent the following passage is, on a scale of 1 to 10:\n\n---\n{new_text}\n---\n\nOnly reply with the number:"
    try:
        response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.8,
            max_tokens=5
        )
        human_like_score = float(response.choices[0].message.content.strip()) / 10
    except:
        human_like_score = 0.5

    # Final reward score
    final = (
        0.4 * length_score +
        0.3 * readability_score +
        0.3 * human_like_score
    )

    return {
        "length_score": round(length_score, 3),
        "readability_score": round(readability_score, 3),
        "human_like_score": round(human_like_score, 3),
        "final_reward": round(final, 3)
    }

# 📌 Step 4: Correct paths as per structure
original_path = "../assets/raw_text/chapter1.txt"
version_path = "../assets/human_edits/human_edit_latest.txt"  # You can change this

# 📌 Step 5: Load both files
with open(original_path, "r", encoding="utf-8") as f:
    original_text = f.read()

with open(version_path, "r", encoding="utf-8") as f:
    new_text = f.read()

# 📌 Step 6: Calculate reward
reward = calculate_reward(original_text, new_text)

print("📊 Reward Scores:")
print(json.dumps(reward, indent=2))

# 📌 Step 7: Save to JSON
os.makedirs("../assets/rewards", exist_ok=True)
output_name = os.path.basename(version_path).replace(".txt", ".json")
output_path = os.path.join("../assets/rewards", output_name)

with open(output_path, "w") as f:
    json.dump(reward, f, indent=2)

print(f"✅ Reward saved at: {output_path}")


📊 Reward Scores:
{
  "length_score": 0.0,
  "readability_score": 0.0,
  "human_like_score": 0.5,
  "final_reward": 0.15
}
✅ Reward saved at: ../assets/rewards/human_edit_latest.json


In [9]:
# 📌 Step 8: RL-Inspired Retry Loop
from datetime import datetime

MAX_TRIES = 3
THRESHOLD = 0.8
attempt = 0
best_reward = 0
best_text = spun

def save_retry_version(text, label):
    path = f"../assets/spun_text/spun_{label}.txt"
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)
    print(f"💾 Saved version: {path}")
    return path

while attempt < MAX_TRIES and best_reward < THRESHOLD:
    print(f"\n🔁 Iteration {attempt + 1}")
    
    # Generate new spin using same writer function
    new_spun = spin_text_llama3(original)
    reward = calculate_reward(original, new_spun)
    final_score = reward["final_reward"]

    print(f"🎯 Reward this iteration: {final_score}")

    # Save if it's better
    if final_score > best_reward:
        best_reward = final_score
        best_text = new_spun
        save_retry_version(best_text, f"best_try{attempt+1}")
        print(f"✅ New Best Reward: {best_reward}")

    attempt += 1

# 📌 Save final best version
save_retry_version(best_text, "final")
print("📘 Final version saved with highest reward.")


NameError: name 'spun' is not defined